<a href="https://colab.research.google.com/github/AbhinavGupta-de/SST_ML/blob/main/Cat_Dog_Classification_Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [58]:
# A random with a seed so that we could reproduce the results.
random.seed(32)

# Create a random 3d Dataset

In [59]:
# Here we are defining some constants
CAT = -1
DOG = 1
N = 100

In [60]:
# Now, we will create some values for both tuna and salmon
def make_random_cats():
  # This is will give us two values
  # First value will be between 0 and 4
  # Second value will be between 0 and 5
  return random.random() * 3, random.random() * 2, random.random() *4

def make_random_dogs():
  # This will give us two values
  # First value will be between 2 and 6
  # Second value will between 2 and 7
  return random.random() * 4 + 2, random.random() * 3 + 3, random.random() * 2 + 3

In [61]:
# Here we are creating an array with different random values
# Example
# cats = [[x11, y11, z11] , [x12, y12, z12], [x13, y13, z13]]
# dogs = [[x21, y21, z21], [x22, y22,z22], [x23,y23, z23]]
cats = [ make_random_cats() for _ in range(N//2) ]
dogs = [ make_random_dogs() for _ in range(N//2) ]

In [62]:
# Now we are creating a dataset
# Example
# dataset = [ ([x1, y1, z1], 0)]
dataset = []
dataset += [ (cat, CAT) for cat in cats ]
dataset += [ (dog, DOG) for dog in dogs ]

In [63]:
# The above thing would have worked for 2d but as we will have to plot in 3d
# We will have to use seaborn to plot and for that we will need to have to make
# our dataframe using pandas


# So, the code below, what it does is creats a dataframe in pandas with data
# data_cats = [{'Height' : x1, 'Weight' : y1, 'Speed': z1, 'Animal': CAT} for x1, y1, z1 in cats]
# df_cats = pd.DataFrame(data_cats)


# The above code would have worked but we directly had the method to implement what I was trying to do

data = [ [*d[0], d[1]] for d in dataset]
df = pd.DataFrame(data)

In [64]:
df.head()

,0,1,2,3
0,0.232265,0.427234,1.212513,-1
1,2.700641,0.992505,2.880962,-1
2,0.300706,1.017854,3.372292,-1
3,1.568398,1.889402,3.527201,-1
4,1.112882,0.001492,3.021069,-1


In [65]:
df.columns = ['Height' , 'Weight', 'Speed', 'Animal']
# animal_col = df['Animal']
# animal_col.loc[animal_col == 0] = 'cat'
# df['Animal'] = animal_col

# animal_col.loc[animal_col == 1] = 'dog'
# df['Animal'] = animal_col

df.head()

,Height,Weight,Speed,Animal
0,0.232265,0.427234,1.212513,-1
1,2.700641,0.992505,2.880962,-1
2,0.300706,1.017854,3.372292,-1
3,1.568398,1.889402,3.527201,-1
4,1.112882,0.001492,3.021069,-1


In [66]:
# getting the value of x axis in one array
# values of y axis in one array and so on for both cats and dogs

cat_x, cat_y, cat_z = zip(*cats)
dog_x, dog_y, dog_z = zip(*dogs)

In [67]:
def plot_model(plane=None):
  #
  # plt.scatter(cat_x, cat_y, cat_z, color='blue', label='Cat')
  # plt.scatter(dog_x, dog_y, dog_z, color='red', label='Dog')

  # plt.title("Cat Dog Clasification")
  # plt.xlabel("Height")
  # plt.ylabel("Weight")
  # plt.k("Speed")

  # if plane is not None:


  # plt.legend()
  # plt.show

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x=cat_x, y=cat_y, z=cat_z, mode='markers', marker=dict(color='blue', size=5), name='Cat'))
  fig.add_trace(go.Scatter3d(x=dog_x, y=dog_y, z=dog_z, mode='markers', marker=dict(color='red', size=5), name='Dog'))

  if plane is not None:
    w, w0 = plane
    w1, w2, w3 = w

    x_values = np.linspace(min(min(cat_x), min(dog_x)), max(max(cat_x), max(dog_x)), 100)
    y_values = np.linspace(min(min(cat_y), min(dog_y)), max(max(cat_y), max(dog_y)), 100)
    x, y = np.meshgrid(x_values, y_values)
    # Calculate the z coordinates of the plane
    z = (-w1 * x - w2 * y - w0) / w3

    # Plot the plane using plot_surface
    fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.5, name='Separating Hyperplane (plane)'))



  # fig = px.scatter_3d(df, x='Height', y='Weight', z='Speed', color='Animal')
  fig.show()

In [68]:
plot_model()

In [ ]:
random_plane = [1, 1,1], -12
plot_model(random_plane)


In [69]:
def norm(w):
  # Return the norm (length) of a vector w
  return abs(sum(wi ** 2 for wi in w) ** 0.5)

def dot(x, w):
  # Return the dot product of two vectors x and w
  return sum(xi * wi for xi, wi in zip(x, w))

def distance_from_plane(x, plane):
  # Return the distance of a point x from a plane defined by a weight vector w and a bias term w0
  w, w0 = plane
  return abs(dot(x, w) + w0) / norm(w)

def loss(data, plane):
  # Return the sum of distances of data points from the plane, weighted by their labels
  return sum(distance_from_plane(xi, plane) * yi for xi, yi in data)


In [70]:
#Bad Model
min_loss = float('inf')
best_plance=None
for w1 in np.arange(-8, 8, 0.51):
  for w2 in np.arange(-8, 8, 0.51):
    for w3 in np.arange(-8, 8, 0.51):
      for w0 in np.arange(-8, 8, 0.51):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

New best plane = ([-8.0, -8.0, -8.0], -8.0) with a loss of 224.3091701531623
New best plane = ([-8.0, -8.0, -8.0], -6.98) with a loss of 224.30917015316228
New best plane = ([-8.0, -8.0, -8.0], -6.470000000000001) with a loss of 224.30917015316226
New best plane = ([-8.0, -8.0, -8.0], -1.8800000000000026) with a loss of 224.3091701531622
New best plane = ([-8.0, -8.0, -8.0], 2.7099999999999955) with a loss of 224.27831613057205
New best plane = ([-8.0, -8.0, -8.0], 3.2199999999999953) with a loss of 224.2047039712504
New best plane = ([-8.0, -8.0, -8.0], 3.729999999999995) with a loss of 224.13109181192874
New best plane = ([-8.0, -8.0, -8.0], 4.239999999999995) with a loss of 224.057479652607
New best plane = ([-8.0, -8.0, -8.0], 4.749999999999995) with a loss of 223.98386749328535
New best plane = ([-8.0, -8.0, -8.0], 5.2599999999999945) with a loss of 223.91025533396362
New best plane = ([-8.0, -8.0, -8.0], 5.769999999999994) with a loss of 223.83664317464198
New best plane = ([-8.0

In [71]:
# Bad Model
min_loss = float('inf')
best_plance=None
for w1 in np.arange(-6, 6, 0.41):
  for w2 in np.arange(-6, 6, 0.41):
    for w3 in np.arange(-6, 6, 0.41):
      for w0 in np.arange(-6, 6, 0.41):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

New best plane = ([-6.0, -6.0, -6.0], -6.0) with a loss of 224.3091701531623
New best plane = ([-6.0, -6.0, -6.0], -5.18) with a loss of 224.30917015316228
New best plane = ([-6.0, -6.0, -6.0], -4.77) with a loss of 224.30917015316217
New best plane = ([-6.0, -6.0, -6.0], -3.539999999999999) with a loss of 224.3091701531621
New best plane = ([-6.0, -6.0, -6.0], 2.200000000000003) with a loss of 224.24608074054225
New best plane = ([-6.0, -6.0, -6.0], 2.610000000000003) with a loss of 224.16717620375297
New best plane = ([-6.0, -6.0, -6.0], 3.020000000000003) with a loss of 224.08827166696375
New best plane = ([-6.0, -6.0, -6.0], 3.4300000000000033) with a loss of 224.00936713017447
New best plane = ([-6.0, -6.0, -6.0], 3.8400000000000034) with a loss of 223.93046259338524
New best plane = ([-6.0, -6.0, -6.0], 4.2500000000000036) with a loss of 223.85155805659602
New best plane = ([-6.0, -6.0, -6.0], 4.660000000000004) with a loss of 223.77265351980682
New best plane = ([-6.0, -6.0, -6.

In [72]:
# Best Model that I have for now
min_loss = float('inf')
best_plance=None
for w1 in np.arange(1, 6, 0.31):
  for w2 in np.arange(1, 6, 0.21):
    for w3 in np.arange(1, 6, 0.41):
      for w0 in np.arange(-10, -0.41, 0.41):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

New best plane = ([1.0, 1.0, 1.0], -10.0) with a loss of -92.16342607054081


In [73]:
# Bad model, intersecting the data
min_loss = float('inf')
best_plance=None
for w1 in np.arange(0.01, 6, 0.31):
  for w2 in np.arange(0.01, 6, 0.21):
    for w3 in np.arange(0.01, 6, 0.41):
      for w0 in np.arange(-10, -0.11, 0.41):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

New best plane = ([0.01, 0.01, 0.01], -10.0) with a loss of -224.3091701531613
New best plane = ([0.01, 0.01, 0.01], -9.59) with a loss of -224.30917015316538
New best plane = ([0.01, 0.01, 0.01], -7.949999999999999) with a loss of -224.30917015316697
New best plane = ([0.32, 0.43, 0.42], -10.0) with a loss of -226.54081314685195
New best plane = ([0.32, 0.43, 0.42], -9.18) with a loss of -226.5408131468521
New best plane = ([0.32, 0.64, 0.42], -10.0) with a loss of -233.40275958121376
New best plane = ([0.32, 0.64, 0.42], -9.18) with a loss of -233.4027595812138
New best plane = ([0.32, 0.64, 0.42], -8.77) with a loss of -233.40275958121384
New best plane = ([0.32, 0.64, 0.42], -7.539999999999999) with a loss of -233.40275958121387
New best plane = ([0.63, 0.85, 0.42], -10.0) with a loss of -234.10216229087288


In [ ]:
# Bad model
min_loss = float('inf')
best_plance=None
for w1 in np.arange(-8, 8, 0.51):
  for w2 in np.arange(-8, 8, 0.51):
    for w3 in np.arange(-8, 8, 0.51):
      for w0 in np.arange(-20, 20, 0.51):
        plane = [w1, w2, w3], w0
        l = abs(loss(dataset, plane))
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

New best plane = ([-8.0, -8.0, -8.0], -20.0) with a loss of 224.30917015316228
New best plane = ([-8.0, -8.0, -8.0], -18.979999999999997) with a loss of 224.30917015316226
New best plane = ([-8.0, -8.0, -8.0], -14.899999999999984) with a loss of 224.3091701531622
New best plane = ([-8.0, -8.0, -8.0], 2.9500000000000703) with a loss of 224.24367511442065
New best plane = ([-8.0, -8.0, -8.0], 3.460000000000072) with a loss of 224.17006295509898
New best plane = ([-8.0, -8.0, -8.0], 3.9700000000000735) with a loss of 224.09645079577732
New best plane = ([-8.0, -8.0, -8.0], 4.480000000000075) with a loss of 224.02283863645562
New best plane = ([-8.0, -8.0, -8.0], 4.990000000000077) with a loss of 223.94922647713392
New best plane = ([-8.0, -8.0, -8.0], 5.500000000000078) with a loss of 223.87561431781228
New best plane = ([-8.0, -8.0, -8.0], 6.01000000000008) with a loss of 223.80200215849055
New best plane = ([-8.0, -8.0, -8.0], 6.520000000000081) with a loss of 223.7283899991689
New best

In [ ]:
# Best model so far
min_loss = float('inf')
best_plance=None
for w1 in np.arange(1, 20, 0.5):
  for w2 in np.arange(1, 20, 0.5):
    for w3 in np.arange(1, 20, 0.5):
      for w0 in np.arange(-12, 0.21, 0.5):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)

In [ ]:
min_loss = float('inf')
best_plance=None
for w1 in np.arange(-2, 5, 0.5):
  for w2 in np.arange(-2, 5, 0.5):
    for w3 in np.arange(-2, 5, 0.5):
      for w0 in np.arange(.001, 21, 0.5):
        plane = [w1, w2, w3], w0
        l = loss(dataset, plane)
        if l < min_loss:
          min_loss = l
          best_plane = plane
          print(f'New best plane = {best_plane} with a loss of {min_loss}')
plot_model(best_plane)